<a href="https://colab.research.google.com/github/brianp0513/NLP-project/blob/main/Hybrid_NLP_Project_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# data preprocessing

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
%matplotlib inline
import time
import re
from wordcloud import WordCloud
import random
import os
from sklearn import metrics 
from scipy.spatial.distance import cdist 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
  
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from collections import OrderedDict
from sklearn.mixture import GaussianMixture
import os
plt.style.use('ggplot')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [124]:
def getTextFromFiles(df, data_path, depression, limit):
    """Return Data Frame """

    for file in os.listdir(data_path)[:limit]:
        with open(data_path + "/" + file, 'r', encoding="ISO-8859-1") as file1:
            file1 = file1.read()
            df = df.append({'text': file1, 'depression': int(depression)}, ignore_index=True)

    return df


In [125]:
def dataPreprocessingForX(df, columnName1):
    df[columnName1] = df[columnName1].map(lambda text: text.lower())
    df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
    stop_words = set(nltk.corpus.stopwords.words('english'))
    df[columnName1] = df[columnName1].map(lambda tokens: [w for w in tokens if not w in stop_words])
    df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))
    df[columnName1] = df[columnName1].map(lambda text: re.sub('[^A-Za-z]+', ' ', text))
    df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
    lemmatizer = WordNetLemmatizer()
    df[columnName1] = df[columnName1].map(lambda text: [lemmatizer.lemmatize(i) for i in text])
    df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))

def dataPreprocessingForY(df, columnName2):
    df[columnName2] = df[columnName2].astype('int32')

In [136]:
data_path_d = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_depression"
data_path_nd = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression"
df = pd.DataFrame(columns=['text', 'depression'])
df = getTextFromFiles(df, data_path_d, 1, 1000)
df = getTextFromFiles(df, data_path_nd, 0, 500)


In [137]:
dataPreprocessingForX(df, 'text')
dataPreprocessingForY(df, 'depression')

In [138]:
X = df['text'].to_numpy()
Y = df['depression'].to_numpy()
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.30)

max_words = 1000
max_len = 150

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


# classifier.fit(counts, Y_train)
# test_counts = count_vectorizer.transform(X_test)
# predictions = classifier.predict(test_counts)

# Gaussian Mixture

In [139]:
def gmm(X_test_gmm):
  y_gmmList = []

  # X =  sequences_matrix
  #Building and fitting the model 
  gmm = GaussianMixture(n_components = 2)
  gmm.fit(X_test_gmm)
  y_gmm = gmm.predict(X_test_gmm)
  y_gmmList.append(y_gmm)
  print(y_gmm.shape)
  return y_gmmList

def gmmClusterResult(y_gmmList, X):
  zero_list = []
  one_list = []
  two_list = []

  for idx,iteration in enumerate(y_gmmList[0]):
    if iteration == 1:
      one_list.append(idx)
    elif iteration == 2:
      two_list.append(idx)
    else:
      zero_list.append(idx)
  # print(zero_list)
  # print(one_list)
  # print(two_list)
  # print(X.shape)
  cluster_zero = []
  cluster_one = []
  cluster_two = []
  for index in zero_list:
    sentence = X[index]
    cluster_zero.append(sentence)
    
  # print(cluster_zero)
  for index in one_list:
    sentence = X[index]
    cluster_one.append(sentence)

  for index in two_list:
    sentence = X[index]
    cluster_two.append(sentence)

  return zero_list, one_list

In [140]:
# print(sequences_matrix)
# print(y_kmList)

In [141]:
def makeWorldCloud(llist):
  depression_words = ''.join(llist)
  depression_wordclod = WordCloud(width = 512,height = 512).generate(depression_words)
  plt.figure(figsize = (10, 8), facecolor = 'k')
  plt.imshow(depression_wordclod)
  plt.axis('off')
  plt.tight_layout(pad = 0)
  plt.show()
# makeWorldCloud(cluster_zero)
# makeWorldCloud(cluster_one)
# makeWorldCloud(cluster_two)

In [142]:
print(test_sequences_matrix.shape)

(450, 150)


In [143]:
y_gmmList_train = gmm(sequences_matrix)
zero_list_train, one_list_train = gmmClusterResult(y_gmmList_train, X_train)
y_gmmList_test = gmm(test_sequences_matrix)
zero_list_test, one_list_test = gmmClusterResult(y_gmmList_test, X_test)

(1050,)
(450,)


# LSTM

In [144]:

sequences_zeroClust = tok.texts_to_sequences(X_train[zero_list_train])
sequences_matrix_zeroClust = sequence.pad_sequences(sequences_zeroClust,maxlen=max_len)
sequences_oneClust = tok.texts_to_sequences(X_train[one_list_train])
sequences_matrix_oneClust = sequence.pad_sequences(sequences_oneClust,maxlen=max_len)

sequences_zeroClust = tok.texts_to_sequences(X_test[zero_list_test])
sequences_matrix_zeroClust_test = sequence.pad_sequences(sequences_zeroClust,maxlen=max_len)
sequences_oneClust = tok.texts_to_sequences(X_test[one_list_test])
sequences_matrix_oneClust_test = sequence.pad_sequences(sequences_oneClust,maxlen=max_len)
# Y_test_LSTM = Y_test[zero_list]

# test_sequences_matrix = sequences_matrix_oneClust
# Y_test = Y[one_list]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_words,50,input_length=max_len))
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))

# model.build(input_shape=sequences_matrix.shape)
# model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

# model.fit(sequences_matrix.astype(float), Y_train.astype(float), batch_size=4,epochs=10,
#           validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

# accr = model.evaluate(test_sequences_matrix, Y_test.astype(float))

# print(" LSTM")
# print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

model.fit(sequences_matrix_zeroClust.astype(float), Y_train[zero_list_train].astype(float), batch_size=4,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

accr = model.evaluate(sequences_matrix_zeroClust_test, Y_test[zero_list_test].astype(float))

print(" LSTM")
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
model.fit(sequences_matrix_oneClust.astype(float), Y_train[one_list_train].astype(float), batch_size=4,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])


accr = model.evaluate(sequences_matrix_oneClust_test, Y_test[one_list_test].astype(float))

print(" LSTM")
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Epoch 1/10
72/72 [==============================] - 1s 14ms/step - loss: 0.6915 - accuracy: 0.5521 - val_loss: 0.6909 - val_accuracy: 0.5342
Epoch 2/10
72/72 [==============================] - 1s 9ms/step - loss: 0.6842 - accuracy: 0.5521 - val_loss: 0.6808 - val_accuracy: 0.5479
Epoch 3/10
72/72 [==============================] - 1s 8ms/step - loss: 0.5958 - accuracy: 0.7257 - val_loss: 0.4438 - val_accuracy: 0.8630
Epoch 4/10
72/72 [==============================] - 1s 8ms/step - loss: 0.2439 - accuracy: 0.9097 - val_loss: 0.3194 - val_accuracy: 0.8356
Epoch 5/10
10/10 [==============================] - 0s 6ms/step - loss: 0.6112 - accuracy: 0.6775
 LSTM
Test set
  Loss: 0.611
  Accuracy: 0.678
Epoch 1/10
138/138 [==============================] - 1s 9ms/step - loss: 0.3333 - accuracy: 0.8802 - val_loss: 0.3250 - val_accuracy: 0.8478
Epoch 2/10
5/5 [==============================] - 0s 4ms/step - loss: 0.5078 - accuracy: 0.8112
 LSTM
Test set
  Loss: 0.508
  Accuracy: 0.811


In [145]:
Y_test[one_list]

array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0], dtyp